### 1nn vs RandomForest

В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

In [2]:
import sklearn.datasets
data = sklearn.datasets.load_digits()

In [12]:
print data.target_names, "\n", data.DESCR

[0 1 2 3 4 5 6 7 8 9] 
Optical Recognition of Handwritten Digits Data Set

Notes
-----
Data Set Characteristics:
    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each block. This generates
an input matrix of 8x8 

Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test

In [28]:
def my_split(x, frac):
    return x[0 : int(x.shape[0] - round(x.shape[0]*frac))], x[- int(round(x.shape[0]*frac)) :]

In [29]:
X_train, X_test = my_split(data.data, 0.25)

In [30]:
X_train.shape

(1348, 64)

In [31]:
X_test.shape

(449, 64)

In [32]:
data.data.shape

(1797, 64)

In [35]:
y_train, y_test = my_split(data.target, 0.25)
print y_train.shape, y_test.shape

(1348,) (449,)


Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

### Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

In [41]:
print X_train[0], y_train[0], "\n", X_train[1], y_train[1]

[  0.   0.   5.  13.   9.   1.   0.   0.   0.   0.  13.  15.  10.  15.   5.
   0.   0.   3.  15.   2.   0.  11.   8.   0.   0.   4.  12.   0.   0.   8.
   8.   0.   0.   5.   8.   0.   0.   9.   8.   0.   0.   4.  11.   0.   1.
  12.   7.   0.   0.   2.  14.   5.  10.  12.   0.   0.   0.   0.   6.  13.
  10.   0.   0.   0.] 0 
[  0.   0.   0.  12.  13.   5.   0.   0.   0.   0.   0.  11.  16.   9.   0.
   0.   0.   0.   3.  15.  16.   6.   0.   0.   0.   7.  15.  16.  16.   2.
   0.   0.   0.   0.   1.  16.  16.   3.   0.   0.   0.   0.   1.  16.  16.
   6.   0.   0.   0.   0.   1.  16.  16.   6.   0.   0.   0.   0.   0.  11.
  16.  10.   0.   0.] 1


In [38]:
from sklearn.metrics.pairwise import euclidean_distances
euclidean_distances(X_train[0], X_train[1])

/home/sigor/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/sigor/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 59.55669568]])

In [44]:
def dist(xn, yn):
    summary = 0
    for x, y in zip(xn, yn):
        summary += (x - y)**2
    return summary

In [48]:
from math import sqrt 
sqrt(dist(X_train[0], X_train[1]))

59.55669567731239

In [49]:
def dist2(xn, yn):
    return sum([(x - y)**2 for x, y in zip(xn, yn)])

In [50]:
sqrt(dist2(X_train[0], X_train[1]))

59.55669567731239

In [58]:
def classify(x, xn_train, yn_train):
    return sorted([[dist(x, x_train), y_train] for x_train, y_train in zip(xn_train, yn_train)])[0]


In [59]:
classify(X_train[1], X_train, y_train)

[0.0, 1]

In [65]:
elems = range(10)
for elem in elems:
    print classify(X_test[elem], X_train, y_train), y_test[elem]

[477.0, 7] 7
[356.0, 3] 3
[290.0, 3] 3
[163.0, 4] 4
[131.0, 6] 6
[197.0, 6] 6
[142.0, 6] 6
[239.0, 4] 4
[134.0, 9] 9
[114.0, 1] 1


Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1

In [68]:
total = errors = 0
for x, y in zip(X_test, y_test):
    if y != classify(x, X_train, y_train)[1]:
        errors += 1
    total += 1
print errors/(total*1.0)

0.0378619153675


### Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. 

In [69]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=1000)

rfc.fit(X_train, y_train)



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [77]:
X_test[1].reshape(1,-1)

array([[  0.,   0.,   5.,  15.,  16.,  15.,   1.,   0.,   0.,  10.,  16.,
         11.,   8.,  16.,   5.,   0.,   0.,  12.,  10.,   1.,  10.,  15.,
          1.,   0.,   0.,   0.,   0.,   8.,  16.,  11.,   1.,   0.,   0.,
          0.,   0.,   1.,  10.,  16.,  10.,   0.,   0.,   0.,   0.,   2.,
          0.,   7.,  16.,   0.,   0.,   0.,   8.,  13.,   5.,  15.,  12.,
          0.,   0.,   0.,   5.,  15.,  16.,  14.,   3.,   0.]])

In [72]:
print rfc.predict(X_test[1])[0], y_test[1]

/home/sigor/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


3 3


In [78]:
total = errors = 0
for x, y in zip(X_test, y_test):
    if y != rfc.predict(x.reshape(1, -1))[0]:
        errors += 1
    total += 1
print errors/(total*1.0)        
    

0.0645879732739


0.0645879732739